<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte3_Java_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%%writefile Server.java
import java.io.IOException;
import java.net.InetAddress;
import java.net.ServerSocket;
import java.net.Socket;
import java.io.DataInputStream;
import java.io.ObjectOutputStream;

public class Server
{
  public static final int port = 4444;
  private static ServerSocket ss = null;

  public static void main(String[] args)
  {
    runServer();
  }

  // Quisimos usar object input stream y no anduvo, cuando leemos se cuelga, PASA CON TODOS
  // write utf me manda cosas null ??
  // buffered reader trae nulls
  public static void runServer()
  {
    Socket socket = null;
    try
    {
      ss = new ServerSocket(port, 5, InetAddress.getLocalHost());

      System.out.println("Socket creado. IP: " + InetAddress.getLocalHost() + " - Puerto: " + port );
      System.out.println("Esperando conexión");
      socket = ss.accept();
      System.out.println("Conexión recibida");

      DataInputStream dataInputStream = new DataInputStream(socket.getInputStream());
      ObjectOutputStream os = new ObjectOutputStream(socket.getOutputStream());
      String input = dataInputStream.readUTF();
      StringInfo info = StringInfo.getStringInfo(input);

      System.out.println("Enviando datos al cliente");
      System.out.println(info);

      os.writeObject(info);
    }
    catch (IOException e)
    {
      System.out.println(e.getStackTrace());
      System.exit(1);
    }
    finally
    {
      closeSockets(socket);
    }
  }

  private static void closeSockets(Socket socket)
  {
    if (ss != null)
    {

      try
      {
        ss.close();
      }
      catch (IOException e)
      {
        throw new RuntimeException(e);
      }
    }
    if (socket != null)
    {
      try
      {
        socket.close();
      }
      catch (IOException e)
      {
        throw new RuntimeException(e);
      }
    }
  }
}

Overwriting Server.java


In [20]:
%%writefile Client.java
import java.io.DataOutputStream;
import java.io.IOException;
import java.io.ObjectInputStream;
import java.net.InetAddress;
import java.net.Socket;

public class Client
{
  public static final int port = 4444;

  public static void main(String[] args)
  {
    Socket socket = null;
    try
    {
      if (args.length < 1)
      {
        System.out.println("Debe indicar una palabra por parametro");
        System.exit(1);
      }

      socket = new Socket(InetAddress.getLocalHost(), port);

      DataOutputStream dataOutputStream = null;
      dataOutputStream = new DataOutputStream(socket.getOutputStream());
      dataOutputStream.writeUTF(args[0]);
      ObjectInputStream is = new ObjectInputStream(socket.getInputStream());
      StringInfo returnMessage = (StringInfo) is.readObject();

      System.out.println(returnMessage);
    }
    catch (IOException e)
    {
      System.out.println("Error en conexión");
      System.out.println(e.getStackTrace());
    }
    catch (ClassNotFoundException e)
    {
      System.out.println("El mensaje recibido no tiene el formato esperado.");
    }
    finally
    {
      if (socket != null)
      {
        try
        {
          socket.close();
        }
        catch (IOException e)
        {
          throw new RuntimeException(e);
        }
      }
    }
  }
}

Overwriting Client.java


In [9]:
%%writefile StringInfo.java
import java.io.Serializable;

public class StringInfo implements Serializable
{
  private int length;
  private int lettersCount;
  private int digitsCount;
  private int specialCharsCount;

  public StringInfo(int length)
  {
    this.length = length;
    lettersCount = 0;
    digitsCount = 0;
    specialCharsCount = 0;
  }

  private void addLetter()
  {
    lettersCount++;
  }

  private void addDigit()
  {
    digitsCount++;
  }

  private void addSpecialChar()
  {
    specialCharsCount++;
  }

  private void processChar(char c)
  {
    if (Character.isLetter(c))
    {
      addLetter();
    }
    else if (Character.isDigit(c))
    {
      addDigit();
    }
    else
    {
      addSpecialChar();
    }
  }

  public static StringInfo getStringInfo(String name)
  {
    StringInfo info = new StringInfo(name.length());
    for (char c : name.toCharArray())
    {
      info.processChar(c);
    }
    return info;
  }

  public int getLength()
  {
    return length;
  }

  public int getLettersCount()
  {
    return lettersCount;
  }

  public int getDigitsCount()
  {
    return digitsCount;
  }

  public int getSpecialCharsCount()
  {
    return specialCharsCount;
  }

  @Override
  public String toString()
  {
    return new StringBuilder()
      .append("Cantidad de caracteres totales: ").append(getLength()).append(System.lineSeparator())
      .append("Cantidad de letras: ").append(getLettersCount()).append(System.lineSeparator())
      .append("Cantidad de dígitos: ").append(getDigitsCount()).append(System.lineSeparator())
      .append("Cantidad de caracteres que no son letra ni dígito: ").append(getSpecialCharsCount())
      .append(System.lineSeparator())
      .toString();
  }
}


Overwriting StringInfo.java


In [21]:
!javac -d . Server.java
!javac -d . Client.java
!javac -d . StringInfo.java

In [22]:
!nohup java Server 1>salidaJava 2>/dev/null &

In [23]:
!java Client "Hol4 munDo 2!"

Cantidad de caracteres totales: 13
Cantidad de letras: 8
Cantidad de dígitos: 2
Cantidad de caracteres que no son letra ni dígito: 3

